In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import comet_ml
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from torch.utils.data import Dataset, random_split, TensorDataset
import numpy as np
import torch

from alegnn.utils import graphTools

%aimport opf
from opf.data import load_data
from opf.gnn import make_gnn
from opf.modules import OPFLogBarrier
from opf.power import load_case, NetworkManager

torch.autograd.set_detect_anomaly(True)

In [3]:
# Constants
data_dir = "../data"
ratio_train = 0.8
dtype = torch.float32
F0 = 2
isjob = False # args.job

# Meta-parameters
A_scaling = 0.001
A_threshold = 0.01
case = "case6ww"

# instantiate the NetworkManager
manager = NetworkManager(load_case(case, data_dir), A_scaling, A_threshold)
N = manager.n_buses

# Parameters / Configuration
param = {
    "train": dict(
        max_epochs=4,
        batch_size=256,
    ),
    "gnn": dict(
        dimLayersMLP=[4 * N],
        dimNodeSignals=[F0, 64],
        nFilterTaps=[4],
    ),
    "barrier": dict(
        type="relaxed_log",  # log, relaxed_log
        t=200,  # higher -> better approximation to barrier
        s=100,  # the slope at which the barrier becomes linear
        cost_weight=1.0,
    ),
    "meta": dict(
        A_scaling=A_scaling,
        A_threshold=A_threshold,
        model="selection",
        case=case
    ),
}

In [12]:
train_samples, test_samples, test_labels = load_data(case, data_dir)
train_data = TensorDataset(torch.from_numpy(train_samples).to(dtype))
test_data = TensorDataset(
    torch.from_numpy(test_samples).to(dtype),
    torch.from_numpy(test_labels).to(dtype)
)

train_split = int(len(train_data) * ratio_train)
val_split = len(train_data) - train_split
train_data, val_data = random_split(train_data, [train_split, val_split])


def collate_fn(batch):
    return tuple([torch.stack(item).to(dtype) for item in zip(*batch)])


train_loader = torch.utils.data.DataLoader(train_data, batch_size=param["train"]["batch_size"], shuffle=True,
                                           collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=param["train"]["batch_size"],
                                         collate_fn=collate_fn)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=param["train"]["batch_size"],
                                          collate_fn=collate_fn)


In [13]:
# Prepare graph shift operator
adjacencyMatrix = manager.get_adjacency()
G = graphTools.Graph("adjacency", adjacencyMatrix.shape[0], {"adjacencyMatrix": adjacencyMatrix})
G.computeGFT()
S, order = graphTools.permDegree(G.S / np.max(np.real(G.E)))  # normalize GSO by dividing by largest e-value

edges = np.sum(S > 0)
print(f"Edges: {edges}")
print(f"Density: {edges / (N * N)}")

Edges: 22
Density: 0.6111111111111112


In [14]:
gnn = make_gnn(S, param["gnn"], dtype=dtype)
barrier = OPFLogBarrier(manager, gnn, **param["barrier"])

In [17]:
logger = CometLogger(workspace="damowerko", project_name="opf", save_dir="./logs")
logger.log_hyperparams(param)
logger.experiment.log_code(folder="../src")
logger.experiment.log_code(folder="../scripts")
logger.experiment.set_model_graph(str(barrier))

CometLogger will be initialized in online mode
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/damowerko/opf/6d2d48c66d2447df865d3e720a7272bd
COMET INFO:   Parameters:
COMET INFO:     barrier/cost_weight : 1.0
COMET INFO:     barrier/s           : 100
COMET INFO:     barrier/t           : 200
COMET INFO:     barrier/type        : relaxed_log
COMET INFO:     gnn/dimLayersMLP    : [24]
COMET INFO:     gnn/dimNodeSignals  : [2, 64]
COMET INFO:     gnn/nFilterTaps     : [4]
COMET INFO:     meta/A_scaling      : 0.001
COMET INFO:     meta/A_threshold    : 0.01
COMET INFO:     meta/case           : case6ww
COMET INFO:     meta/model          : selection
COMET INFO:     train/batch_size    : 256
COMET INFO:     train/max_epochs    : 2
COMET INFO:   Uploads [count]:
COMET INFO:     environment details 

In [18]:
trainer = pl.Trainer(progress_bar_refresh_rate=(0 if isjob else 1), gpus=1, max_epochs=param["train"]["max_epochs"])
trainer.fit(barrier, train_loader, val_loader)
logger.experiment.end()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type         | Params
--------------------------------------
0 | gnn  | SelectionGNN | 9.8 K 
--------------------------------------
9.8 K     Trainable params
0         Non-trainable params
9.8 K     Total params
0.039     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

c:\users\damow\.virtualenvs\opf-1s7-svgk-py3.8\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
c:\users\damow\.virtualenvs\opf-1s7-svgk-py3.8\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/damowerko/opf/abd2d19944c0447f836b4f0f2ab84996
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [7] : (-512.1500854492188, -490.6253662109375)
COMET INFO:   Parameters:
COMET INFO:     barrier/cost_weight : 1.0
COMET INFO:     barrier/s           : 100
COMET INFO:     barrier/t           : 200
COMET INFO:     barrier/type        : relaxed_log
COMET INFO:     gnn/dimLayersMLP    : [24]
COMET INFO:     gnn/dimNodeSignals  : [2, 64]
COMET INFO:     gnn/nFilterTaps     : [4]
COMET INFO:     meta/A_scaling      : 0.001
COMET INFO:     meta/A_threshold    : 0.01
COMET INFO:     meta/case           : case6ww
COMET INFO:     meta/model          : selection
COMET INFO:     train/batch_size    : 256
COMET INFO:     train/max_epochs    : 2
COME

In [21]:
trainer.test(test_dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_cost': -520.4771118164062,
 'test_equality_loss': 1.035457730293274,
 'test_equality_powerflow_loss': 1.0000000830368325e-16,
 'test_equality_reactive_power_loss': 0.0,
 'test_equality_real_power_loss': 0.0,
 'test_equality_reference_loss': 0.07080059498548508,
 'test_equality_voltage_angle_difference_loss': nan,
 'test_equality_voltage_magnitude_loss': 0.9646571278572083,
 'test_inequality_backward_rate_max_loss': -0.01897253841161728,
 'test_inequality_backward_rate_max_violated_rate': 0.0,
 'test_inequality_backward_rate_max_violated_rms': 0.0,
 'test_inequality_forward_rate_max_loss': -0.019049545750021935,
 'test_inequality_forward_rate_max_violated_rate': 0.0,
 'test_inequality_forward_rate_max_violated_rms': 0.0,
 'test_inequality_loss': 176.9461212158203,
 'test_inequality_reactive_power_max_loss': -0.023026946932077408,
 'test_inequality_reactive_power_max_violated

[{'test_cost': -520.4771118164062,
  'test_equality_loss': 1.035457730293274,
  'test_inequality_loss': 176.9461212158203,
  'test_equality_powerflow_loss': 1.0000000830368325e-16,
  'test_equality_reference_loss': 0.07080059498548508,
  'test_equality_real_power_loss': 0.0,
  'test_inequality_real_power_min_loss': -0.01577027514576912,
  'test_inequality_real_power_min_violated_rate': 0.0,
  'test_inequality_violated_rate': 1.015059232711792,
  'test_inequality_real_power_min_violated_rms': 0.0,
  'test_inequality_violated_rms': 2.3160526752471924,
  'test_inequality_real_power_max_loss': 83.76557922363281,
  'test_inequality_real_power_max_violated_rate': 0.010448264889419079,
  'test_inequality_real_power_max_violated_rms': 1.07327401638031,
  'test_equality_reactive_power_loss': 0.0,
  'test_inequality_reactive_power_min_loss': -0.023024694994091988,
  'test_inequality_reactive_power_min_violated_rate': 0.0,
  'test_inequality_reactive_power_min_violated_rms': 0.0,
  'test_inequali